In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r'D:\Cdrive files\FSDS\kaggle\XGBoost + k-fold CV + Feature Importance\Wholesale customers data.csv')
df.shape

(440, 8)

In [3]:
X = df.drop('Channel', axis=1)

y = df['Channel']

In [4]:
X.head()

,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,3,12669,9656,7561,214,2674,1338
1,3,7057,9810,9568,1762,3293,1776
2,3,6353,8808,7684,2405,3516,7844
3,3,13265,1196,4221,6404,507,1788
4,3,22615,5410,7198,3915,1777,5185


In [5]:
y.head()

0    2
1    2
2    2
3    1
4    2
Name: Channel, dtype: int64

In [6]:
# convert labels into binary values

y[y == 2] = 0

y[y == 1] = 1

In [7]:
# again preview the y label

y.head()

0    0
1    0
2    0
3    1
4    0
Name: Channel, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [11]:
!pip install hyperopt

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------  1.6/1.6 MB 13.9 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 12.1 MB/s  0:00:00

   ---------------------------------------- 0/3 [py4j]
   ---------------------------------------- 0/3 [py4j]
   ---------------------------------------- 0/3 [py4j]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 

In [12]:
from hyperopt import hp

space = {
    'max_depth': hp.quniform("max_depth", 3, 18, 1),
    'gamma': hp.uniform('gamma', 1, 9),
    'reg_alpha': hp.quniform('reg_alpha', 40, 180, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'min_child_weight': hp.quniform('min_child_weight', 0, 10, 1),
    'n_estimators': 180,
    'seed': 0
}

In [13]:
def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [16]:
from hyperopt import fmin, tpe, hp, Trials

# Define the search space
space = {
    'max_depth': hp.quniform("max_depth", 3, 18, 1),
    'gamma': hp.uniform('gamma', 1, 9),
    'reg_alpha': hp.quniform('reg_alpha', 40, 180, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'min_child_weight': hp.quniform('min_child_weight', 0, 10, 1),
    'n_estimators': 180,
    'seed': 0
}

# Define your objective function
def objective(params):
    # For demonstration, let's just return a dummy loss
    return {'loss': params['gamma'], 'status': 'ok'}

# Create a Trials object to store results
trials = Trials()

# Run hyperparameter optimization
best_hyperparams = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trials
)

print("Best hyperparameters:", best_hyperparams)

100%|██████████████████████████████████████████████| 100/100 [00:01<00:00, 50.15trial/s, best loss: 1.0087021128796128]
Best hyperparameters: {'colsample_bytree': 0.6171227860032026, 'gamma': 1.0087021128796128, 'max_depth': 7.0, 'min_child_weight': 7.0, 'reg_alpha': 175.0, 'reg_lambda': 0.22263075091454082}


In [17]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

100%|██████████████████████████████████████████████| 100/100 [00:01<00:00, 53.75trial/s, best loss: 1.0037405209323393]
